## 作业3：气动系数的可视化
对典型的二维和三维气动数据表CLBasic进行数据库存储设计，通过Sqlite数据库进行存取。设计线性插值函数进行插值运算，提取任意工作点的气动系数。设计气动数据表的可视化界面，对二维气动系数进行曲线绘制，对三维气动系数进行曲面绘制，曲线和曲面采用样条函数进行平滑。


![pic1.png](pic1.png)
![pic2.png](pic2.png)
![pic3.png](pic3.png)

数据结构及基础设置

In [4]:
import sys
from PyQt5.QtWidgets import QApplication, QWidget, QVBoxLayout, QListWidget, QLabel
from PyQt5.QtCore import Qt
from bidict import bidict
import sqlite3
import numpy as np

# 使用numpy不显示科学计数e
np.set_printoptions(suppress=True)

# 气动系数类型列表
coefficient_category_list = ["CL", "CD", "CY", "Cm", "Cl", "Cn"]

# 数据库表名称 双向字典(双向映射)
table_bidict = bidict({
    "clb_2_0_7": 0,
    "cdaab_3_0_5": 1,
    "cdmal_3_0_8": 2,
    "cdmah_3_0_9": 3,
    "cybeta_7_0_5_1": 4,
    "cmaab_4_0_8and9": 5,
    "cmaeie_4_0_21": 6,
    "cmaeoe_4_0_22": 7,
    "crs_5_0_7": 8,
    "cria_5_0_23": 9,
    "cnbeta_6_0_6_1": 10,
    "cnrudue_6_0_20": 11,
    "cnrudle_6_0_21": 12
})

# 气动系数名称 双向字典(双向映射)
coefficient_bidict = bidict({
    "CL_Basic": 0,
    "CD_Basic": 1,
    "CD_M_flap_up": 2,
    "CD_M_flap_down": 3,
    "CY_beta": 4,
    "Cm_Basic": 5,
    "Cm_ele_in": 6,
    "Cm_ele_out": 7,
    "Cl_beta_low": 8,
    "Cl_ail_in": 9,
    "Cn_beta": 10,
    "Cn_rud_up": 11,
    "Cn_rud_low": 12
})

# COEFFICIENT_NODE 类
class COEFFICIENT_NODE:
    def __init__(self, table, coefficient):
        self.table = table
        self.coefficient = coefficient
        self.field = []
        self.data = []

    def __str__(self):
        return ("table: {} | coefficient: {} | data: {}".format(self.table, self.coefficient, self.data))

从数据库文件读入数据

In [5]:
# 初始化dataset
dataset = []
for i in range(13):
    temp = COEFFICIENT_NODE(table_bidict.inverse[i], coefficient_bidict.inverse[i])
    dataset.append(temp)

# 读取数据库文件
database_file = "B747data0.db"
connect = sqlite3.connect(database_file)
cursor = connect.cursor()

# 检验数据库文件是否有损坏或数据缺失
sql = "SELECT name FROM sqlite_master WHERE type='table';"
cursor.execute(sql)
database_table = [item[0] for item in cursor.fetchall()]
for datanode in dataset:
    if not datanode.table in database_table:
        raise Exception("错误! 数据库文件损坏或数据丢失!")

# 从数据库文件读取数据并放入dataset数据集
for datanode in dataset:
    sql = "SELECT * FROM " + datanode.table
    cursor.execute(sql)
    database_data = [list(item) for item in cursor.fetchall()]
    for j in range(len(database_data[0])):
        temp = []
        datanode.field.append(database_data[0][j])
        for i in range(1, len(database_data)):
            temp.append(database_data[i][j])
        datanode.data.append(temp)



PyQt5 & 主程序

In [6]:
import sys
from PyQt5 import QtWidgets, QtCore, QtGui
from matplotlib.backends.backend_qt5agg import FigureCanvasQTAgg as FigureCanvas
from matplotlib.figure import Figure
import numpy as np
from scipy.interpolate import splrep, splev

# 线性插值
def linear_interpolate(x, x_points, y_points):
    for i in range(1, len(x_points)):
        if x_points[i-1] <= x <= x_points[i]:
            y = y_points[i-1] + (y_points[i] - y_points[i-1]) * (x - x_points[i-1]) / (x_points[i] - x_points[i-1])
            return y
    return None  # 超出可插值范围

# 全局变量，记录当前选择的第三个 ListBox 的数据
current_selection = None

class PlotCanvas(FigureCanvas):
    def __init__(self, parent=None):
        fig = Figure()
        self.ax = fig.add_subplot(111)
        super().__init__(fig)
        self.setParent(parent)
        self.lines = []
        self.original_linewidths = []
        self.show_htx()

    def plot(self, x_data, y_data_list, labels):
        self.ax.clear()
        self.lines = []
        self.original_linewidths = []
        for y_data, label in zip(y_data_list, labels):
            spline = splrep(x_data, y_data)
            x_smooth = np.linspace(np.min(x_data), np.max(x_data), 500)
            y_smooth = splev(x_smooth, spline)
            line, = self.ax.plot(x_smooth, y_smooth, label=label)
            self.lines.append(line)
            self.original_linewidths.append(line.get_linewidth())
        self.ax.legend()
        self.ax.grid(True, which='both', linestyle='--', linewidth=0.5, color='gray', alpha=0.7)  # 添加方格坐标线
        self.ax.minorticks_on()  # 添加刻度线
        self.draw()

    def highlight_line(self, label):
        for line, original_width in zip(self.lines, self.original_linewidths):
            if line.get_label() == label:
                line.set_linewidth(3)  # 高亮线宽
            else:
                line.set_linewidth(original_width)  # 恢复默认线宽
        self.draw()

    def show_htx(self):
        self.ax.clear()
        self.ax.text(0.5, 0.5, 'HTX', horizontalalignment='center', verticalalignment='center', 
                     transform=self.ax.transAxes, fontsize=50, color='black', alpha=0.9)
        self.ax.axis('off')  # 隐藏刻度线和方格坐标线
        self.draw()

class App(QtWidgets.QWidget):
    def __init__(self):
        super().__init__()
        self.initUI()

    def initUI(self):
        # main_layout
        main_layout = QtWidgets.QVBoxLayout()

        # Canvas & matplotlib
        self.canvas = PlotCanvas(self)
        main_layout.addWidget(self.canvas)

        # 输入输出文本框
        input_output_layout = QtWidgets.QHBoxLayout()
        
        self.input_label = QtWidgets.QLabel("输入自变量:")
        input_output_layout.addWidget(self.input_label)
        self.input_box = QtWidgets.QLineEdit()
        self.input_box.setValidator(QtGui.QDoubleValidator())
        input_output_layout.addWidget(self.input_box)

        self.output_label = QtWidgets.QLabel("插值计算结果:")
        input_output_layout.addWidget(self.output_label)
        self.output_box = QtWidgets.QLineEdit()
        self.output_box.setReadOnly(True)
        input_output_layout.addWidget(self.output_box)

        main_layout.addLayout(input_output_layout)

        # listbox 选择数据集
        listbox_layout = QtWidgets.QHBoxLayout()

        self.listbox_1 = QtWidgets.QListWidget()
        self.listbox_1.addItems(coefficient_category_list)
        self.listbox_1.itemSelectionChanged.connect(self.handle_first_listbox_selection)
        listbox_layout.addWidget(self.listbox_1)

        self.listbox_2 = QtWidgets.QListWidget()
        self.listbox_2.itemSelectionChanged.connect(self.handle_second_listbox_selection)
        listbox_layout.addWidget(self.listbox_2)

        self.listbox_3 = QtWidgets.QListWidget()
        self.listbox_3.itemSelectionChanged.connect(self.handle_third_listbox_selection)
        listbox_layout.addWidget(self.listbox_3)

        main_layout.addLayout(listbox_layout)

        # 按钮
        button_layout = QtWidgets.QHBoxLayout()
        
        self.calculate_button = QtWidgets.QPushButton("计算插值结果")
        self.calculate_button.clicked.connect(self.calculate)
        button_layout.addWidget(self.calculate_button)

        self.reset_button = QtWidgets.QPushButton("重置")
        self.reset_button.clicked.connect(self.reset)
        button_layout.addWidget(self.reset_button)

        main_layout.addLayout(button_layout)

        self.setLayout(main_layout)
        self.setWindowTitle('B747-100气动系数可视化及插值计算工具 by HTX')
        self.show()

    def handle_first_listbox_selection(self):
        global current_selection
        current_selection = None
        self.reset()  # 重置一切
        self.update_second_listbox()  # 更新第二个listbox

    def handle_second_listbox_selection(self):
        global current_selection
        current_selection = None
        self.output_box.clear()
        self.update_third_listbox_and_plot()

    def handle_third_listbox_selection(self):
        self.output_box.clear()
        global current_selection
        selected_label_item = self.listbox_3.currentItem()
        if selected_label_item:
            selected_label = selected_label_item.text()
            current_selection = (selected_label, self.listbox_2.currentItem().text())
            self.canvas.highlight_line(selected_label)

    def update_second_listbox(self):
        selection = self.listbox_1.currentItem()
        if selection is not None:
            selection_text = selection.text()
            self.listbox_2.clear()
            for node in dataset:
                if node.coefficient.startswith(selection_text):
                    self.listbox_2.addItem(str(node.coefficient))  # 确保是字符串

    def update_third_listbox_and_plot(self):
        selection = self.listbox_2.currentItem()
        if selection is not None:
            selection_text = selection.text()
            self.listbox_3.clear()
            for node in dataset:
                if node.coefficient == selection_text:
                    self.listbox_3.addItems([str(field) for field in node.field[1:]])  # 确保是字符串
                    x_data = node.data[0]
                    y_data_list = node.data[1:]
                    labels = node.field[1:]
                    self.canvas.plot(x_data, y_data_list, labels)
                    break

    def calculate(self):
        """计算线性插值并显示在self.output_box 使用self.output_box.setText"""
        if current_selection is None:
            self.output_box.setText("未选择数据")
            return
        try:
            input_value = float(self.input_box.text())
            selected_label, selected_coefficient = current_selection
            for node in dataset:
                if node.coefficient == selected_coefficient:
                    selected_label = float(selected_label)
                    for i in range(len(node.field)):
                        if node.field[i]==selected_label:
                            y_index = i
                            break
                    y_points = node.data[y_index]
                    x_points = node.data[0]
                    result = linear_interpolate(input_value, x_points, y_points)
                    # Output result
                    if result == None:
                        self.output_box.setText("超出可插值数据范围")
                    else:
                        self.output_box.setText(str(result))
                    break
        except ValueError:
            self.output_box.setText("输入有误")
        except Exception as e:
            self.output_box
            self.output_box.setText(f"错误: {e}")

    def reset(self):
        self.input_box.clear()
        self.output_box.clear()
        self.listbox_1.clearSelection()
        self.listbox_2.clear()
        self.listbox_3.clear()
        self.canvas.show_htx()

if __name__ == '__main__':
    app = QtWidgets.QApplication(sys.argv)
    ex = App()
    sys.exit(app.exec_())


SystemExit: 0